# Jupyter QueryMaster

In [1]:
import requests
import json

In [2]:
def soql_query(instance_url, acccess_token, q):

    headers = {'Authorization' : 'Bearer ' + acccess_token,
               'Content-Type' : 'application/json'
              }
    
    r = requests.get(instance_url + '/services/data/v41.0/query/?q=' + q,
                    headers = headers)

    
    soql_response = json.loads(r.text)
    records = soql_response['records']
    print('Is it all?' , soql_response['done'], ', of total', soql_response['totalSize'])    
    return soql_response

In [15]:
def insert_record(instance_url, objectType, acccess_token, body):
    
    headers = {'Authorization' : 'Bearer ' + acccess_token,
           'Content-Type' : 'application/json'
          }

    r = requests.post(instance_url + '/services/data/v41.0/sobjects/' + objectType + '/',
                 headers = headers,
                 data = json.dumps(body))
    
    return json.loads(r.text)



In [4]:
def get_oauth_token(OAUTH_LOGIN_URL, CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD):
    d = {'grant_type' : 'password',
        'client_id' : CLIENT_ID,
        'client_secret' : CLIENT_SECRET,
         'username' : USERNAME,
         'password' : PASSWORD
        }

    r = requests.post(OAUTH_LOGIN_URL, 
                      data = d)

    response_map = json.loads(r.text)
    acccess_token = response_map['access_token']
    instance_url = response_map['instance_url']

    return acccess_token, instance_url

In [5]:
# get the oAuth token

OAUTH_LOGIN_URL = 'https://test.salesforce.com/services/oauth2/token'
CLIENT_ID = '3MVG9szVa2RxsqBaVMAZFSLdWv5BZvhnb15ChwBZzbVVyb_u9j08M.xnlQ5Ly15mI6KZfVRTsjTQSMsY27rDT'
CLIENT_SECRET = '5979639566859185104'
USERNAME = 'blackthorn@prismhr.test'
PASSWORD = '2P0kf3FLMu22YkFTJCMcH0VXkFgV7C17Eu6dG'

In [6]:
acccess_token, instance_url = get_oauth_token(OAUTH_LOGIN_URL, CLIENT_ID, CLIENT_SECRET, USERNAME, PASSWORD)
acccess_token

'00D3D0000000TFV!AQ4AQIPMD5b.KxVJXfaT1qtvpzgU2D8to0uzwth6ZTq._COk1ToB_.ZRcQZrZinGQTvzixNRNMNU.JxCfjTV4Ty46b6wvOnq'

In [11]:
## convertint the tiered prices to tiers
productId = '01t3D0000012VR2';

Q = "SELECT Name, Product__c, billing360__Lower_Tier__c, billing360__Upper_Tier__c, billing360__Type__c, billing360__Price__c FROM billing360__Tiered_Price__c WHERE Product__c = '" + productId + "'"
tieredPriceList = soql_query(instance_url, acccess_token, Q)['records']

ppBody = {'Product__c' : productId,
       'Name' : 'pp' + productId,
       'billing360__Currency_ISO__c' : 'USD',
       'billing360__Default__c' : True,
       'billing360__Active__c' : True}

pp = insert_record(instance_url, 'billing360__Price_Plan__c',acccess_token, ppBody)

for tieredPrice in tieredPriceList:
    tierBody = {
        'billing360__Lower_Tier__c' : tieredPrice['billing360__Lower_Tier__c'],
        'billing360__Upper_Tier__c' : tieredPrice['billing360__Upper_Tier__c'],
        'billing360__Price_Plan__c' : pp['Id'],
        'billing360__Type__c' : tieredPrice['billing360__Type__c']
    }
    
    tier = insert_record(instance_url, 'billing360__Tier__c',acccess_token, tierBody)
    print('tier ok', tier['Id'])

Is it all? True , of total 3


'a8B3D00000004n2UAA'

{'Name': '1001-2000', 'billing360__Type__c': 'Tiered', 'Product__c': '01t3D0000012VR2QAM', 'billing360__Price__c': 0.6, 'attributes': {'url': '/services/data/v41.0/sobjects/billing360__Tiered_Price__c/a7U3D0000004CUBUA2', 'type': 'billing360__Tiered_Price__c'}, 'billing360__Lower_Tier__c': 1001.0, 'billing360__Upper_Tier__c': 2000.0}
{'Name': '2001-5000', 'billing360__Type__c': 'Tiered', 'Product__c': '01t3D0000012VR2QAM', 'billing360__Price__c': 0.5, 'attributes': {'url': '/services/data/v41.0/sobjects/billing360__Tiered_Price__c/a7U3D0000004CUGUA2', 'type': 'billing360__Tiered_Price__c'}, 'billing360__Lower_Tier__c': 2001.0, 'billing360__Upper_Tier__c': 5000.0}
{'Name': '1-1000', 'billing360__Type__c': 'Tiered', 'Product__c': '01t3D0000012VR2QAM', 'billing360__Price__c': 0.75, 'attributes': {'url': '/services/data/v41.0/sobjects/billing360__Tiered_Price__c/a7U3D0000004CULUA2', 'type': 'billing360__Tiered_Price__c'}, 'billing360__Lower_Tier__c': 1.0, 'billing360__Upper_Tier__c': 1000.

In [130]:
body = {'Name' : 'jaaaa', 'billing360__Lower_Tier__c' : 199}
r = insert_record(instance_url, 'billing360__Tiered_Price__c', acccess_token, body)

In [131]:
r

<Response [201]>

In [132]:
r.text

'{"id":"a7U3D0000004D1GUAU","success":true,"errors":[],"warnings":[]}'